### We can easily collect available information by utilizing selenium library on python. You also need chromedriver.exe file that is compatible with the Google Chrome Version you are currently using (other browsers may require different solutions):


In [ ]:
import numpy as np
import pandas as pd
import selenium as sel
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## FLOW:

Here, I'd like to make a simple web scrapping tool for myself to collect different locations of a Telecom company. The example cae is just randomly selected. The simple tool I have coded below can be extended and enhanced, and also, it might not work if some changes have occured for the related web-site.

##### Base URL:

In [ ]:
url = "https://sa.zain.com/en/locate-us"

##### Connection:

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.get(url)

##### For dropdown menu:

- find_element_by_name or by_xnext could also be tried. A properly written xnext should be the most prone to alterations:

In [ ]:
regions = Select(driver.find_element_by_id('map-regions'))
sub_regions = Select(driver.find_element_by_id('map-subregions'))

##### For Clicking

In [ ]:
element = WebDriverWait(driver,1).until(EC.presence_of_element_located((By.ID,"locateus-search")))

#### The Loop for Creating the Dictionary:

Here, I examined the organization of the website and arranged my code accordingly to store information in a dictionary. Later on, I will transform this into a dataframe to save.

In [ ]:
genel ={}
region_len = len(regions.options)

for reg in range(1,3):

    regions.select_by_index(reg)
    element.click()
    reg_name = regions.first_selected_option.text
    
    sub_dict={}
    new_option_len = len(sub_regions.options)
    for sub in range(0,new_option_len):

        sub_regions.select_by_index(sub)
        element.click()
        sub_name = sub_regions.first_selected_option.text
        print(sub_name)
        
        stores = driver.find_elements_by_class_name('store-title')
        store_list =''
        try:
            for i in range(1,len(stores)):
                store_list += str(stores[i].text) + '/'
        except:
            continue
        
        sub_dict[sub_name] = store_list
            
    genel[reg_name] = sub_dict

##### From Dict To Dataframe & Excel:

In [ ]:
empty = pd.DataFrame(columns=['Region','Sub-Region','Store Types'])
for r, s in genel.items():
    
    data = pd.DataFrame(list(s.items()),columns=['Sub-Region','Store Types'])
    data['Region']=r
    empty=empty.append(data)

In [ ]:
empty.reset_index(drop=True).to_excel('Zain Locations.xlsx')